Counting bigrams leading up to QUERY and after QUERY

In [1]:
import json

query = 'probably'

#load json file of probablies
with open('app/json_files/combined/probably.json') as json_data:
	parsed_json = json.load(json_data)

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

#define stopwords
punctuation = list(string.punctuation)
digits = list(string.digits)
extra = [query, '\'s', '\'\'', '``', '--', '...', 'n\'t', '\'d', '\'ll', '\'re', '\'m']
stop = stopwords.words('english') + punctuation + digits + extra

Start by looking at the words that comes after "probably" in each sentence, taking out NLTK's English stopwords.

In [52]:
#def nextwords

def find_nextwords(parsed_json, stop):
    next_words = []
    missing_query_list = []
    for item in parsed_json:
        title = word_tokenize(item['title'].lower())
        try:
            query_index = title.index(query)
            try:
                next_word = title[query_index+1]
            except ValueError:
                missing_query_list.append(item['title'])
                continue
                
        if next_word not in stop:
            next_words.append(next_word)
    return next_words, missing_query_list

next_words, missing_query_list = find_nextwords(parsed_json,stop)

SyntaxError: invalid syntax (<ipython-input-52-da053171c01b>, line 16)

In [4]:
#count nextwords
from collections import Counter

def count_common(word_list):
    count_all= Counter()
    count_all.update(word_list)
    print(len(word_list))
    print(count_all.most_common(25))

count_common(next_words)

543
[('taken', 30), ('congressional', 16), ('near', 12), ('used', 12), ('ca', 11), ('march', 10), ('end', 9), ('original', 7), ('orville', 6), ('new', 6), ('salem', 6), ('never', 5), ('part', 5), ('detroit', 5), ('michigan', 5), ('1920', 5), ('early', 4), ('going', 4), ('hungarian', 4), ('c.', 4), ('photographed', 4), ('installed', 4), ('south', 4), ('added', 4), ('around', 4)]


However, it turns out stopwords like "in" and "at" are kind of interesting here. So let's look at the same set of titles with a different set of stopwords, this time including prepositions.

In [5]:
prepositions = ['in', 'at', 'on']
new_stopwords = [ w for w in stop if w not in prepositions]
no_stopwords = extra + punctuation + digits + extra

In [6]:
next_with_prepositions, missing_query_list = find_nextwords(parsed_json, new_stopwords)
next_no_stop, missing_query_list = find_nextwords(parsed_json, no_stopwords)

print(count_common(next_with_prepositions))
print(count_common(next_no_stop))

790
[('in', 179), ('at', 52), ('taken', 30), ('on', 16), ('congressional', 16), ('near', 12), ('used', 12), ('ca', 11), ('march', 10), ('end', 9), ('original', 7), ('orville', 6), ('new', 6), ('salem', 6), ('never', 5), ('part', 5), ('detroit', 5), ('michigan', 5), ('1920', 5), ('early', 4), ('going', 4), ('hungarian', 4), ('c.', 4), ('photographed', 4), ('installed', 4)]
None
1106
[('in', 179), ('the', 61), ('a', 54), ('at', 52), ('taken', 30), ('from', 28), ('for', 22), ('during', 20), ('be', 17), ('on', 16), ('congressional', 16), ('of', 14), ('not', 14), ('under', 13), ('near', 12), ('used', 12), ('ca', 11), ('march', 10), ('end', 9), ('original', 7), ('orville', 6), ('to', 6), ('new', 6), ('about', 6), ('his', 6)]
None


Next let's look at what else happens after the query, by creating bigrams (or trigrams?) from each title after the word probably

In [61]:
def find_phrases(parsed_json):
    next_phrases = []
    missing_query_list = []
    last_phrases = []
    
    for item in parsed_json:
        title = word_tokenize(item['title'].lower())
        try:
            query_index = title.index(query)
        except ValueError:
            missing_query_list.append(item['title'])
            continue
            
        if query_index == len(title)-1:
            last_phrase = title[:query_index]
            last_phrases.append(title)
        else:
            next_phrase = title[query_index:]
            next_phrases.append(next_phrase)
    
    return next_phrases, missing_query_list, last_phrases

next_phrases, missing_query_list, last_phrases = find_phrases(parsed_json)

[['minnesota', '--', 'probably'], ['sen.', 'ford', '/', 'rep.', 'charlie', 'probably']]


Let's look at what's before the probably. It's really only interesting if it's the last word or in parathenses.

In [47]:
last_phrases = []
for item in parsed_json:
        title = word_tokenize(item['title'].lower())
        try:
            query_index = title.index(query)
            if query_index >= len(title)-1:
                last_phrase = title[:query_index+1]
                last_phrases.append(title)
        except ValueError:
            continue

print (last_phrases)

[['minnesota', '--', 'probably'], ['sen.', 'ford', '/', 'rep.', 'charlie', 'probably']]


So then we can sort based on the first word of each phrase